### Install Libraries & Dependency

In [1]:
!pip install --upgrade pip

In [2]:
!pip install protobuf==3.20.3
!pip install absl-py rouge_score nltk
!pip install numpy

  Preparing metadata (setup.py) ... - \ done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=430e0c15fda44daac00920ff29cb25f73db2d42ef803f417cee13f3ac92c5e1f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
!pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [4]:
!pip install protobuf==3.20.3

In [5]:
!pip install absl-py

In [6]:
!pip install rouge_score 

In [7]:
!pip install rouge

In [8]:
!pip install nltk

In [9]:
!pip install sentencepiece

In [10]:
!pip install datasets

In [11]:
!pip install numpy

In [12]:
!pip install evaluate 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.5 MB/s eta 0:00:00


In [13]:
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


Check if GPU is available and make it enable

In [14]:
import torch
import pandas as pd

if torch.cuda.is_available():
    print("GPU is enabled.")
    print("device count: {}, current device: {}".format(torch.cuda.device_count(), torch.cuda.current_device()))
else:
    print("GPU is not enabled.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU is enabled.
device count: 1, current device: 0


### Data Preparation

Using created dataset for maroon team

In [15]:
from datasets import load_dataset

ds = load_dataset("bgspaditya/maroon100k")
ds

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/bgspaditya--maroon100k-f8cb1371f4070783/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['article', 'summary'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['article', 'summary'],
        num_rows: 10000
    })
})

checking data 

In [16]:
ds["train"][0]

{'article': 'Selama musim mudik dan arus balik Idul Fitri 2017, truk tertentu dilarang melintas di ruas jalan tertentu. Termasuk masuk ke jalan tol melalui gerbang di Cikarang, Bekasi, Jawa Barat. Namun, pada akhir pekan ini kendaraan besar yang membawa barang diperbolehkan kembali masuk ke gerbang tol Cikarang. " Truk baru boleh masuk tol Cikarang hari Minggu ( 2 / 6 ), " kata Kapolda Metro Jaya, Irjen M. Iriawan, saat meninjau arus balik di Cikarang, Jumat ( 30 / 6 ). Iriawan menegaskan, selama masa larangan diberlakukan, truk yang coba-coba melintas di jalan tol pasti diberhentikan. " Pasti diberhentikan sebelum masuk, cuma dikoordinasikan dulu sama ( Dinas ) Perhubungan Darat, " sebutnya. Sebagai informasi, mulai H - 4 Idul Fitri pemerintah memberlakukan larangan bagi truk tertentu untuk melalui jalur yang dilalui pemudik. Larangan tersebut tidak berlaku untuk truk pembawa bahan makanan. Truk sembako yang boleh melintas saat Lebaran juga dibatasi, yaitu hanya truk dengan maksimal d

### Preprocessing

Using Tokenizer from multilingualT5-small

In [17]:
from transformers import AutoTokenizer

t5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


#### Tokenize data

In [18]:
def tokenize_sample_data(data):
    # Max token size is 14536 and 215 for inputs and labels, respectively.
    # Here I restrict these token size.
    input_feature = t5_tokenizer(data["article"], truncation=True, max_length=1024)
    label = t5_tokenizer(data["summary"], truncation=True, max_length=128)
    return {
        "input_ids": input_feature["input_ids"],
        "attention_mask": input_feature["attention_mask"],
        "labels": label["input_ids"],
    }


In [19]:
tokenized_ds = ds.map(
    tokenize_sample_data,
    remove_columns=["summary", "article"],
    batched=True,
    batch_size=128)
tokenized_ds

  0%|          | 0/625 [00:00<?, ?ba/s]

  0%|          | 0/79 [00:00<?, ?ba/s]

  0%|          | 0/79 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})

#### Load Model

In [20]:
from transformers import AutoConfig, AutoModelForSeq2SeqLM

mt5_config = AutoConfig.from_pretrained(
    "google/mt5-small",
    max_length=128,
    length_penalty=0.6,
    no_repeat_ngram_size=2,
    num_beams=15,
)
model = (AutoModelForSeq2SeqLM
         .from_pretrained("google/mt5-small", config=mt5_config)
         .to(device))

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [21]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    t5_tokenizer,
    model=model,
    return_tensors="pt")

#### Define Evaluation Function

In [22]:
!pip install mxnet-mkl==1.6.0 numpy==1.23.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 60.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.4.1 requires cupy

In [23]:
!pip install numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

def tokenize_sentence(arg):
    encoded_arg = t5_tokenizer(arg)
    return t5_tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    preds, labels = eval_arg
    # Replace -100
    labels = np.where(labels != -100, labels, t5_tokenizer.pad_token_id)
    # Convert id tokens to text
    text_preds = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Insert a line break (\n) in each sentence for ROUGE scoring
    # (Note : Please change this code, when you perform on other languages except for Japanese)
    text_preds = [(p if p.endswith(("!", "！", "?", "？", "。")) else p + "。") for p in text_preds]
    text_labels = [(l if l.endswith(("!", "！", "?", "？", "。")) else l + "。") for l in text_labels]
    sent_tokenizer_jp = RegexpTokenizer(u'[^!！?？。]*[!！?？。]')
    text_preds = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(p))) for p in text_preds]
    text_labels = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(l))) for l in text_labels]
    # compute ROUGE score with custom tokenization
    return rouge_metric.compute(
        predictions=text_preds,
        references=text_labels,
        tokenizer=tokenize_sentence
    )

Test evaluation on sample data

In [25]:
from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
    tokenized_ds["test"].with_format("torch"),
    collate_fn=data_collator,
    batch_size=5)
for batch in sample_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"].to(device),
            num_beams=15,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
            max_length=128,
        )
    labels = batch["labels"]
    break

metrics_func([preds, labels])

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


{'rouge1': 0.12847227386529833,
 'rouge2': 0.09172870140612077,
 'rougeL': 0.12847227386529833,
 'rougeLsum': 0.12726423902894496}

### Modeling

#### Tuning

In [26]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir = "/kaggle/working/mt5-summarize-id",
    log_level = "error",
    num_train_epochs = 3,
    learning_rate = 5e-4,
    lr_scheduler_type = "linear",
    warmup_steps = 90,
    optim = "adafactor",
    weight_decay = 0.01,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 16,
    evaluation_strategy = "steps",
    eval_steps = 100,
    predict_with_generate=True,
    generation_max_length = 128,
    save_steps = 500,
    logging_steps = 10,
    push_to_hub = False
)

In [27]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["validation"].select(range(20)),
    tokenizer = t5_tokenizer,
)

In [28]:
pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


#### Training

In [29]:
trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_23/4032920361.py:1 in <module>                                                    │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_23/4032920361.py'                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1662 in train                    │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1845 in _inner_training_loop     │
│                                                                                                  │
│   1842 │   │   self._globalstep_last_logged = self.state.global_step                             │
│   1843 │   │   model.zero_grad()                                                                 │
│   1844 │   │                                                                                     │
│ ❱ 1845 │   │   self.control = self.callback_handler.on_train_begin(args, self.state, self.contr  │
│   1846 │   │                                                                                     │
│   1847 │   │   # Skip the first epochs_trained epochs to get the random state of the dataloader  │
│   1848 │   │   if not args.ignore_data_skip:                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer_callback.py:353 in on_train_begin   │
│                                                                                                  │
│   350 │                                                                                          │
│   351 │   def on_train_begin(self, args: TrainingArguments, state: TrainerState, control: Trai   │
│   352 │   │   control.should_training_stop = False                                               │
│ ❱ 353 │   │   return self.call_event("on_train_begin", args, state, control)                     │
│   354 │                                                                                          │
│   355 │   def on_train_end(self, args: TrainingArguments, state: TrainerState, control: Traine   │
│   356 │   │   return self.call_event("on_train_end", args, state, control)                       │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer_callback.py:397 in call_event       │
│                                                                                                  │
│   394 │                                                                                          │
│   395 │   def call_event(self, event, args, state, control, **kwargs):                           │
│   396 │   │   for callback in self.callbacks:              

#### Saving Fine-Tuned Model

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# /kaggle/working/

In [ ]:
import os

os.makedirs("/kaggle/working/trained_for_summarization_id", exist_ok=True)
if hasattr(trainer.model, "module"):
    trainer.model.module.save_pretrained("/kaggle/working/trained_for_summarization_id")
else:
    trainer.model.save_pretrained("/kaggle/working/trained_for_summarization_id")

In [ ]:
# import os

# os.makedirs("/content/drive/MyDrive/trained_for_summarization_id", exist_ok=True)
# if hasattr(trainer.model, "module"):
#     trainer.model.module.save_pretrained("/content/drive/MyDrive/trained_for_summarization_id")
# else:
#     trainer.model.save_pretrained("/content/drive/MyDrive/trained_for_summarization_id")

### Using Fine-Tuned Model

Load Model

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = (AutoModelForSeq2SeqLM
         .from_pretrained("/kaggle/working/trained_for_summarization_id")
         .to(device))

In [ ]:
model

In [ ]:
!zip -r model.zip '/kaggle/working/trained_for_summarization_id'

In [ ]:
from IPython.display import FileLink
FileLink(r'model.zip')

#### Testing Model

Test on sample data

In [ ]:
from torch.utils.data import DataLoader
sample_dataloader = DataLoader(
    tokenized_ds["test"].with_format("torch"),
    collate_fn=data_collator,
    batch_size=5)
for batch in sample_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"].to(device),
            num_beams=15,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
            max_length=128,
        )
    labels = batch["labels"]
    break
labels = np.where(labels != -100, labels, t5_tokenizer.pad_token_id)
text_preds = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
text_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)
# Show result
print("***** Input's Text *****")
print(ds["test"]["article"][0])
print("***** Summary Text (True Value) *****")
print(text_labels[0])
print("***** Summary Text (Generated Text) *****")
print(text_preds[0])

In [ ]:
print("***** Input's Text *****")
print(ds["test"]["article"][2])
print("***** Summary Text (True Value) *****")
print(text_labels[2])
print("***** Summary Text (Generated Text) *****")
print(text_preds[2])

Test on An Example Article

In [ ]:
ARTICLE_TO_SUMMARIZE = "Still Life, Vase with Daisies, and Poppies dibuat beberapa bulan sebelum van Gogh meninggal. Still Life, Vase with Daisies, and Poppies melampaui perkiraan sebelumnya US$50 juta pada pelelangan di Sotheby. Lukisan Van Gogh dibuat di rumah dokter Prancisnya hanya beberapa bulan sebelum dirinya meninggal di tahun 1890. Karyanya ini adalah satu dari beberapa lukisannya yang terjual saat Van Gogh masih hidup. Rekor penjualan pelukis Belanda pada balai lelang adalah US$82,5 juta lewat Portrait of Dr Gachet. Lukisan tentang dokternya ini terjual pada tahun 1990. Pada pelelangan ini patung Alberto Giacometti dari tahun1951 terjual seharga lebih dari US$100 juta, di bawah rekor karya seniman Swiss itu senilai US$104,3 juta. Sementara pematung Amedeo Modigliani mencatat rekor baru lewat Tete patung kepala yang dibuat di tahun 1911-1912 dari batu kapur terjual lebih US$70 juta."
TRUE_SUMM = "Sebuah mahakarya Vincent van Gogh, yang dilukis pada akhir kehidupannya, terjual di New York sebesar US$61,8 juta atau Rp752 miliar."

In [ ]:
input_ids = t5_tokenizer.encode(ds["test"]["article"][0], return_tensors='pt', truncation=True, max_length=1024)
label = t5_tokenizer.encode(text_labels[0], truncation=True, return_tensors='pt', max_length=128)

In [ ]:
# THIS MODEL
with torch.no_grad():
  preds = model.generate(
              input_ids.to(device),
              num_beams=15,
              num_return_sequences=1,
              no_repeat_ngram_size=1,
              remove_invalid_values=True,
              max_length=128,
          )

In [ ]:
text_preds1 = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
print(text_preds1[0])

### Evaluation

In [ ]:
eval_model_owned = metrics_func([preds, label])
eval_model_owned

#### Comparing Model

In [ ]:
# CAHYA MODEL
summary_ids = model.generate(input_ids.to(device),
            min_length=20,
            max_length=80,
            num_beams=10,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.8,
            top_k = 50,
            top_p = 0.95)

In [ ]:
summary_text = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

In [ ]:
eval_model_cahya = metrics_func([summary_ids, label])
eval_model_cahya

Compare ROUGE Score

In [ ]:
evals = [eval_model_owned, eval_model_cahya]
rouge_scores = pd.concat([pd.Series(d) for d in evals], axis=1).fillna(0).T
rouge_scores.index = ['Owned T5 Model', 'Cahya Model']
rouge_scores

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge
ROUGE = Rouge()

In [ ]:
ROUGE.get_scores(summary_text, TRUE_SUMM)

In [ ]:
ROUGE.get_scores(text_preds1[0], TRUE_SUMM)

In [ ]:
ROUGE.get_scores(text_preds1[0], TRUE_SUMM)

In [ ]:
# input_ids = t5_tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors='pt', truncation=True, max_length=1024)
# with torch.no_grad():
#   preds = model.generate(
#               input_ids.to(device),
#               num_beams=15,
#               num_return_sequences=1,
#               no_repeat_ngram_size=1,
#               remove_invalid_values=True,
#               max_length=128,
#           )
# text_preds1 = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
# print(text_preds1[0])